In [7]:
from collections import deque
from threading import Thread, Lock
from time import sleep

# Bai 1: kiem tra du ngoac trong bieu thuc

In [5]:
def is_valid(sequence):
    # kiem tra ngoai le
    if len(sequence) == 0:
        return

    # khai bao 1 stack de giu cac ngoac da xet
    stack = deque()
    # duyet qua tung phan tu trong vong lap
    for par in sequence:
        if par == "(": # open par
            stack.append(par)
        elif par == ")": # close par
            if len(stack) > 0:
                stack.pop() # remove open par matching with close par
            else:
                return False # du dau dong ngoac / dau dong ngoac nam truoc mo ngoac
    
    return len(stack) == 0 # neu = 0 => khong bi du dau mo ngoac

In [ ]:
print(is_valid("(((((())))))")) # true
print(is_valid("(3*2) + (10*(12+4)")) #false

# Bai 2: To chuc hang doi 

In [ ]:
class Customer:
    def __init__(self, cashier):
        self.cashier = cashier

    def create_order(self, order_name):
        order_id = self.cashier.receive_order(order_name)
        print("Order #{}: {} recorded.".format(order_id, order_name))

In [ ]:
class Cashier:
    def __init__(self):
        self.order_queue = deque()  # tao danh sach luu tru don da dat
        self.order_count = 0

    def set_chef(self, chef):
        self.chef = chef

    def receive_order(self, order_name):
        self.order_count += 1  # cong don so thu tu cho danh sach
        if not self.chef.receive_request((self.order_count, order_name)):
            # kiem tra neu chef khong phuc vu => nau mon moi
            self.order_queue.append((self.order_count, order_name))

        return self.order_count  # tra so cho khach hang

    def service_order(self, order):
        print("==> Order #{}: {} served.".format(order[0], order[1]))
        
        # kiem tra va add don tiep theo vao hang doi phuc vu
        if len(self.order_queue) > 0 and self.chef.receive_request((self.order_queue[0])):
            self.order_queue.popleft() # da phuc vu xong => out

In [ ]:
class Chef:
    def __init__(self, cashier):
        self.__lock = Lock()  # bien private (khong co quyen truy cap ngoai class)
        self.cashier = cashier

    def receive_request(self, order):
        busy = self.__lock.locked()
        if not busy:
            thread = Thread(target=self.__make_order, args=(order))  # tao luong
            thread.start()  # chay luong
        return not busy

    # ham private (ngoai class Chef khong duoc goi den)
    def __make_order(self, order):
        self.__lock.acquire()
        sleep(1)  # moi mon phuc vu trong 1 giay
        self.__lock.release()
        self.cashier.service_order(order)

In [ ]:
# init objects
cashier = Cashier()
chef = Chef(cashier)
cashier.set_chef(chef)
customer = Customer(cashier)

# simulation
customer.order("Vietnamese Coffee")
customer.order("Capuchino")
sleep(3)
customer.order("Latte")
sleep(2)
customer.order("Mocha")